In [1]:
import pandas as pd
import logging
import glob

In [2]:
# Assign data file path
data_file='../../assets/data/samples/query_181021_partial.csv'

sample_df = pd.read_csv(data_file, nrows=100)

sample_df.head(5)

,Unnamed: 0,score,favorite_count,view_count,answer_count,link,title,body,tags
0,0,5,NaN,1711,4,https://stackoverflow.com/questions/2577911/c-...,C++ vector pointer/reference problem,<p>Please take a look at this example:</p>\n\n...,c++|vector|pointers
1,1,5,NaN,13285,3,https://stackoverflow.com/questions/3069448/ho...,How to declare a void pointer in C#,<p>How can we declare a void pointer in C#?</p>,c#|c#-4.0|void
2,2,5,NaN,14437,1,https://stackoverflow.com/questions/17144405/s...,SSIS OLEDB destination with SQL command,<p>I have a simple SSIS package with a data fl...,sql|sql-server-2008|ssis|oledb
3,3,5,NaN,158,5,https://stackoverflow.com/questions/48735618/i...,Is this the correct way of writing an array of...,"<p>I have a class ""Employee""\nI want to create...",c++
4,4,5,NaN,842,1,https://stackoverflow.com/questions/29913935/c...,Can I enable hardware acceleration for Angular...,<p>The current version does not seem smooth an...,angularjs|hardware-acceleration|angular-material


In [3]:
%reload_ext autoreload
%autoreload 2
from ktext.preprocess import processor

Using TensorFlow backend.


In [4]:
sample_body_raw = sample_df.body.tolist()
sample_body_raw[0]

'<p>Please take a look at this example:</p>\n\n<pre><code>#include &lt;iostream&gt;\n#include &lt;vector&gt;\n#include &lt;string&gt;\nusing namespace std;\n\nclass mySubContainer\n{\npublic:\n    string val;\n};\n\nclass myMainContainer\n{\npublic:\n    mySubContainer sub;\n};\n\nvoid doSomethingWith( myMainContainer &amp;container )\n{\n    container.sub.val = "I was modified";\n}\n\nint main( )\n{\n    vector&lt;myMainContainer&gt; vec;\n\n\n    /**\n     * Add test data\n     */\n    myMainContainer tempInst;\n\n    tempInst.sub.val = "foo";\n    vec.push_back( tempInst );\n\n    tempInst.sub.val = "bar";\n    vec.push_back( tempInst );\n\n\n    // 1000 lines of random code here\n\n    int i;\n    int size = vec.size( );\n    myMainContainer current;\n\n    for( i = 0; i &lt; size; i ++ )\n    {\n        cout &lt;&lt; i &lt;&lt; ": Value before=\'" &lt;&lt; vec.at( i ).sub.val &lt;&lt; "\'" &lt;&lt; endl;\n\n        current = vec.at( i );\n        doSomethingWith( current );\n\n   

In [5]:
body_pp = processor(keep_n=8000, padding_maxlen=70)
sample_body_vecs = body_pp.fit_transform(sample_body_raw)

In [6]:
print('\noriginal string:\n', sample_body_raw[0], '\n')
print('after pre-processing:\n', sample_body_vecs[0], '\n')


original string:
 <p>Please take a look at this example:</p>

<pre><code>#include &lt;iostream&gt;
#include &lt;vector&gt;
#include &lt;string&gt;
using namespace std;

class mySubContainer
{
public:
    string val;
};

class myMainContainer
{
public:
    mySubContainer sub;
};

void doSomethingWith( myMainContainer &amp;container )
{
    container.sub.val = "I was modified";
}

int main( )
{
    vector&lt;myMainContainer&gt; vec;


    /**
     * Add test data
     */
    myMainContainer tempInst;

    tempInst.sub.val = "foo";
    vec.push_back( tempInst );

    tempInst.sub.val = "bar";
    vec.push_back( tempInst );


    // 1000 lines of random code here

    int i;
    int size = vec.size( );
    myMainContainer current;

    for( i = 0; i &lt; size; i ++ )
    {
        cout &lt;&lt; i &lt;&lt; ": Value before='" &lt;&lt; vec.at( i ).sub.val &lt;&lt; "'" &lt;&lt; endl;

        current = vec.at( i );
        doSomethingWith( current );

        cout &lt;&lt; i &lt;&lt; ": Value

In [7]:
' '.join([body_pp.id2token[idx] for idx in sample_body_vecs[0] if idx >1])

'p please take a look at this example p pre code include lt iostream gt include lt vector gt include lt string gt using namespace std class mysubcontainer public string val class mymaincontainer public mysubcontainer sub void dosomethingwith mymaincontainer amp container container sub val i was modified int main vector lt mymaincontainer gt vec add test data mymaincontainer tempinst tempinst sub val foo vec push back tempinst tempinst sub val'

#### HTML Not Properly Cleaned
Bring in Beautiful Soup

In [8]:
from bs4 import BeautifulSoup

In [95]:
def strip_example_html(text, strip_code=True):
    soup = BeautifulSoup(text, "html5lib")
    if strip_code:
        for code in soup("code"):
            code.decompose()
    return soup.get_text()

def strip_list_html(text_list, strip_code=True):
    soup_list = [strip_example_html(text, strip_code=strip_code) for text in text_list]
    
    return soup_list

In [39]:
sample_body_soup = strip_list_html(sample_body_raw)
sample_body_soup[0]

"Please take a look at this example:\n\n\n\nA hell lot of code for an example, I know.\n\nNow so you don't have to spend years thinking about what this [should] do[es]: I have a class myMainContainer which has as its only member an instance of mySubContainer. mySubContainer only has a string val as member.\n\nSo I create a vector and fill it with some sample data.\n\nNow, what I want to do is: Iterate through the vector and make a separate function able to modify the current myMainContainer in the vector. However, the vector remains unchanged as the output tells: \n\n\n\n\nWhat am I doing wrong?\n\n\ndoSomethingWith has to return void, I can't let it return the modified myMainContainer and then just overwrite it in the vector, that's why I tried to pass it by reference as seen in the doSomethingWith definition above."

In [57]:
body_pp2 = processor(keep_n=8000, padding_maxlen=70)
sample_body_vecs2 = body_pp2.fit_transform(sample_body_soup)

In [58]:
print('\noriginal string:\n', sample_body_soup[0], '\n')
print('after pre-processing:\n', sample_body_vecs2[0], '\n')


original string:
 Please take a look at this example:



A hell lot of code for an example, I know.

Now so you don't have to spend years thinking about what this [should] do[es]: I have a class myMainContainer which has as its only member an instance of mySubContainer. mySubContainer only has a string val as member.

So I create a vector and fill it with some sample data.

Now, what I want to do is: Iterate through the vector and make a separate function able to modify the current myMainContainer in the vector. However, the vector remains unchanged as the output tells: 




What am I doing wrong?


doSomethingWith has to return void, I can't let it return the modified myMainContainer and then just overwrite it in the vector, that's why I tried to pass it by reference as seen in the doSomethingWith definition above. 

after pre-processing:
 [178 340   5 618  56  12  77   5 965 267  10  42  14  18  77   3  59 134
  24  84  70  19  16   4 966 967 341 135  39  12  78  29 968   3  16   5


In [53]:
print('after pre-processing:\n', sample_body_vecs2[4], '\n')

after pre-processing:
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   2 344
 202  73  30 121 987   7 988 276   5 147  68 989  21   5 453 637 454   2
 990   6  11  86   2 455 638   6  30 991  32  51  11   2 455  10   2 992
  44  30 456 639  31   3  40 455 638  29   3  16   4  90   2 993] 



In [59]:
' '.join([body_pp2.id2token[idx] for idx in sample_body_vecs2[0] if idx >1])

'please take a look at this example a hell lot of code for an example i know now so you don t have to spend years thinking about what this should do es i have a class mymaincontainer which has as its only member an instance of mysubcontainer mysubcontainer only has a string val as member so i create a vector and fill it with some sample data now what'

#### Prepocessing successful.

Now to determine number of keepwords and padding chars.

Using partial dataset for analysis.

In [61]:
df = pd.read_csv(data_file)
df.size

222273

Clean HTML before analysis

In [64]:
df_body_raw = df.body.tolist()
df_title_raw = df.title.tolist()

In [96]:
df_body_souped = strip_list_html(df_body_raw, strip_code=False)

In [97]:
df_souped = pd.DataFrame({
    'title': df_title_raw,
    'body': df_body_souped
})
pd.set_option('display.max_colwidth', -1)
df_souped.head(3)

,title,body
0,C++ vector pointer/reference problem,"Please take a look at this example:\n\n#include <iostream>\n#include <vector>\n#include <string>\nusing namespace std;\n\nclass mySubContainer\n{\npublic:\n string val;\n};\n\nclass myMainContainer\n{\npublic:\n mySubContainer sub;\n};\n\nvoid doSomethingWith( myMainContainer &container )\n{\n container.sub.val = ""I was modified"";\n}\n\nint main( )\n{\n vector<myMainContainer> vec;\n\n\n /**\n * Add test data\n */\n myMainContainer tempInst;\n\n tempInst.sub.val = ""foo"";\n vec.push_back( tempInst );\n\n tempInst.sub.val = ""bar"";\n vec.push_back( tempInst );\n\n\n // 1000 lines of random code here\n\n int i;\n int size = vec.size( );\n myMainContainer current;\n\n for( i = 0; i < size; i ++ )\n {\n cout << i << "": Value before='"" << vec.at( i ).sub.val << ""'"" << endl;\n\n current = vec.at( i );\n doSomethingWith( current );\n\n cout << i << "": Value after='"" << vec.at( i ).sub.val << ""'"" << endl;\n }\n system(""pause"");//i suck\n\n}\n\n\nA hell lot of code for an example, I know.\n\nNow so you don't have to spend years thinking about what this [should] do[es]: I have a class myMainContainer which has as its only member an instance of mySubContainer. mySubContainer only has a string val as member.\n\nSo I create a vector and fill it with some sample data.\n\nNow, what I want to do is: Iterate through the vector and make a separate function able to modify the current myMainContainer in the vector. However, the vector remains unchanged as the output tells: \n\n0: Value before='foo'\n0: Value after='foo'\n1: Value before='bar'\n1: Value after='bar'\n\n\n\nWhat am I doing wrong?\n\n\ndoSomethingWith has to return void, I can't let it return the modified myMainContainer and then just overwrite it in the vector, that's why I tried to pass it by reference as seen in the doSomethingWith definition above."
1,How to declare a void pointer in C#,How can we declare a void pointer in C#?
2,SSIS OLEDB destination with SQL command,"I have a simple SSIS package with a data flow task with ADO.NET source & destination:\n\n\n\nNormally, I would just select 'table or view' on the OLEDB destination, but although the connection is correct it's not showing me any table names. \n\nSo: I have to use SQL command to insert rows in the OLE DB destination. That destination is a DB2 database.\n\nMy question: how can I write this sql statement to insert everything from the ANO NET source into the destination table?\n"


In [98]:
df_len['body_len'] = df_souped['body'].str.len()
df_len['title_len'] = df_souped['title'].str.len()
df_len.head(3)

,body_len,title_len
0,1835,36
1,40,35
2,469,39


In [99]:
df_len.describe()

,body_len,title_len
count,24697.000000,24697.000000
mean,1076.607280,52.104183
std,1507.114857,19.412864
min,28.000000,12.000000
25%,371.000000,38.000000
50%,670.000000,50.000000
75%,1223.000000,63.000000
max,30556.000000,150.000000


Look at number fo unique words in souped body sample

In [100]:
unique_words = list(set(" ".join(df_body_souped).split(" ")))
len(unique_words)

494881

Text Processor Analysis

In [111]:
body_proc = processor(hueristic_pct_padding=.9, keep_n=150000)

In [113]:
body_vecs = body_proc.fit_transform(df_body_souped)

 See full histogram by insepecting the `document_length_stats` attribute.


In [115]:
body_stats = body_proc.document_length_stats
body_stats.head(75)

,bin,doc_count,cumsum_pct
230,5,11,0.000445
1,10,121,0.005345
39,15,238,0.014982
6,20,277,0.026198
23,25,432,0.043690
29,30,529,0.065109
10,35,661,0.091874
38,40,651,0.118233
51,45,739,0.148156
19,50,735,0.177916


Check top tokens

In [119]:
token_count_df = body_proc.token_count_pandas()
token_count_df.head(10)

,count
number,157506
the,137332
i,115942
to,94063
a,72519
is,54469
in,48517
and,47306
this,39207
of,38357


Start first run with padding_maxlen=250 (~ 85 percentile) and keep_n=150000 (processor default)